# Imports pour le scraping et la sérialisation

In [2]:
from dataclasses import dataclass
from requests import get
from bs4 import BeautifulSoup
from serde import serialize, deserialize
from serde.json import to_json, from_json

# Exploration de l'API BeautifulSoup

- Pour [les types d'objets composant l'arbre](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#kinds-of-objects)
- Pour [les façons d'explorer l'arbre](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#navigating-the-tree)

In [3]:
adresse = "https://fr.wikipedia.org/wiki/CAC_40"

In [16]:
# Requête http et vérification
page = get(adresse)
page.status_code

200

In [17]:
# "Analyse grammaticale"
code = page.content.decode("utf8")
soupe = BeautifulSoup(code, "lxml")

In [6]:
type(soupe)

bs4.BeautifulSoup

In [7]:
# Récupération de la liste des balises enfants
cs = list(soupe.children)
len(cs)

2

In [8]:
gauche, droite = cs

In [9]:
type(gauche)

bs4.element.Doctype

In [10]:
gauche.name

In [11]:
gauche

'html'

In [12]:
type(droite)

bs4.element.Tag

In [13]:
# type de balise
droite.name

'html'

In [19]:
# accès par attribut objet python aux première balises enfant d'un type donné
droite.head

<head>
<meta charset="utf-8"/>
<title>CAC 40 — Wikipédia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janvier","février","mars","avril","mai","juin","juillet","août","septembre","octobre","novembre","décembre"],"wgRequestId":"b2dc0b71-df55-452a-be5c-d7880aab7667","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"CAC_40","wgTitle":"CAC 40","wgCurRevisionId":187118135,"wgRevisionId":187118135,"wgArticleId":649,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Page utilisant P571","Page utilisant P946","Article utilisant l'infobox Indice boursier","Article utilisant une Infobox","Article à référence souhaitée","Article à référence nécessaire","Catégorie Commons avec lien local identique sur Wikidata","Arti

In [20]:
# dictionnaire des attributs (élements dans  la balise outre le nom)
droite.body.attrs

{'class': ['mediawiki',
  'ltr',
  'sitedir-ltr',
  'mw-hide-empty-elt',
  'ns-0',
  'ns-subject',
  'mw-editable',
  'page-CAC_40',
  'rootpage-CAC_40',
  'skin-vector',
  'action-view',
  'skin-vector-search-vue']}

In [21]:
# Rercherche d'un type de balise
tables = soupe.find_all(name="table")
len(tables)

8

In [22]:
# Recherche d'un type de balise avec filtre sur l'attribut class
nouvel_essai = soupe.find_all(name="table", attrs={"class": ["wikitable", "sortable"]})
len(nouvel_essai)

3

In [23]:
for tbl in nouvel_essai:
    print(tbl.attrs)

{'class': ['wikitable', 'sortable']}
{'border': '3', 'class': ['wikitable', 'sortable']}
{'cellpadding': '2', 'cellspacing': '2', 'class': ['wikitable', 'sortable', 'gauche'], 'style': 'text-align: center; font-size: 90%;'}


In [24]:
ma_table = nouvel_essai[0]

## Exercice

1. Récupérer une liste des lignes de la table html
2. Récupérer la liste des entreprises avec leur chiffre d'affaire et leur capitalisation. (liste de triplets)
3. Ordonner les sociétés par ordre décroissant de capitalisation.

In [26]:
ma_table.thead

In [27]:
lignes = ma_table.tbody.find_all("tr")
len(lignes)

41

In [28]:
header, *lignes = lignes

In [29]:
header

<tr>
<th width="15%">Société<sup class="reference" id="cite_ref-valeur_58-0"><a href="#cite_note-valeur-58"><span class="cite_crochet">[</span>26<span class="cite_crochet">]</span></a></sup>
</th>
<th width="20%">Secteur
</th>
<th data-sort-type="number" width="9%">Poids indiciel au 28/10/2020 (en %)<sup class="reference" id="cite_ref-59"><a href="#cite_note-59"><span class="cite_crochet">[</span>27<span class="cite_crochet">]</span></a></sup>
</th>
<th data-sort-type="number" width="9%"><a href="/wiki/Chiffre_d%27affaires" title="Chiffre d'affaires">Chiffre d'affaires</a> 2019 (en millions d'euros)
</th>
<th data-sort-type="number" width="9%"><a href="/wiki/Capitalisation_boursi%C3%A8re" title="Capitalisation boursière">Capitalisation boursière</a> au 08/09/2020<sup class="reference" id="cite_ref-Cap_60-0"><a href="#cite_note-Cap-60"><span class="cite_crochet">[</span>28<span class="cite_crochet">]</span></a></sup> (en milliards d'euros)
</th>
<th width="9%">Entrée dans l'indice
</th>

In [30]:
lignes[0]

<tr>
<td><a href="/wiki/Air_liquide" title="Air liquide">Air liquide</a>
</td>
<td><a href="/wiki/Gaz_industriel" title="Gaz industriel">Gaz industriel</a>
</td>
<td>4,04
</td>
<td>21 920
</td>
<td>66,84
</td>
<td><span data-sort-value="19871231 !"></span><time class="nowrap date-lien" data-sort-value="1987-12-31" datetime="1987-12-31"><a href="/wiki/31_d%C3%A9cembre" title="31 décembre">31</a> <a href="/wiki/D%C3%A9cembre_1987" title="Décembre 1987">décembre</a> <a href="/wiki/1987" title="1987">1987</a></time>
</td></tr>

In [31]:
# affichage formaté pour plus de lisibilité
print(lignes[0].prettify())

<tr>
 <td>
  <a href="/wiki/Air_liquide" title="Air liquide">
   Air liquide
  </a>
 </td>
 <td>
  <a href="/wiki/Gaz_industriel" title="Gaz industriel">
   Gaz industriel
  </a>
 </td>
 <td>
  4,04
 </td>
 <td>
  21 920
 </td>
 <td>
  66,84
 </td>
 <td>
  <span data-sort-value="19871231 !">
  </span>
  <time class="nowrap date-lien" data-sort-value="1987-12-31" datetime="1987-12-31">
   <a href="/wiki/31_d%C3%A9cembre" title="31 décembre">
    31
   </a>
   <a href="/wiki/D%C3%A9cembre_1987" title="Décembre 1987">
    décembre
   </a>
   <a href="/wiki/1987" title="1987">
    1987
   </a>
  </time>
 </td>
</tr>



In [32]:
# Extraction des informations voulues
# sous forme structurée
@dataclass
class Resume:
    societe: str
    chiffre_affaire: int
    capitalisation_boursiere: int


resultat = list()
for ligne in lignes:
    societe, secteur, indice, ca, cb, date = ligne.find_all("td")
    societe, *_ = societe.strings
    ca, *_ = ca.strings
    cb, *_ = cb.strings
    resultat.append(
        Resume(
            societe, 
            int(ca.strip().replace(" ", "")) * 10**6, 
            int(10 ** 9 * float(cb.strip().replace(",", ".")))
        )
    )
    
resultat

[Resume(societe='Air liquide', chiffre_affaire=21920000000, capitalisation_boursiere=66840000000),
 Resume(societe='Airbus', chiffre_affaire=70478000000, capitalisation_boursiere=57840000000),
 Resume(societe='Alstom', chiffre_affaire=8072000000, capitalisation_boursiere=10540000000),
 Resume(societe='ArcelorMittal', chiffre_affaire=61737000000, capitalisation_boursiere=11790000000),
 Resume(societe='Atos', chiffre_affaire=12258000000, capitalisation_boursiere=7860000000),
 Resume(societe='Axa', chiffre_affaire=103532000000, capitalisation_boursiere=41850000000),
 Resume(societe='BNP Paribas', chiffre_affaire=44597000000, capitalisation_boursiere=46740000000),
 Resume(societe='Bouygues', chiffre_affaire=37929000000, capitalisation_boursiere=13060000000),
 Resume(societe='Capgemini', chiffre_affaire=14125000000, capitalisation_boursiere=20050000000),
 Resume(societe='Carrefour', chiffre_affaire=80672000000, capitalisation_boursiere=11680000000),
 Resume(societe='Crédit agricole', chiffr

In [33]:
# Tri via l'utilisation des attributs et le sort naïf
resultat.sort(key=lambda res: res.capitalisation_boursiere, reverse=True)
resultat

[Resume(societe='LVMH', chiffre_affaire=53670000000, capitalisation_boursiere=271410000000),
 Resume(societe="L'Oréal", chiffre_affaire=29873000000, capitalisation_boursiere=155820000000),
 Resume(societe='Sanofi', chiffre_affaire=36126000000, capitalisation_boursiere=107680000000),
 Resume(societe='TotalEnergies', chiffre_affaire=175133000000, capitalisation_boursiere=89080000000),
 Resume(societe='Hermès International', chiffre_affaire=6883000000, capitalisation_boursiere=78060000000),
 Resume(societe='Kering', chiffre_affaire=15883000000, capitalisation_boursiere=70840000000),
 Resume(societe='Air liquide', chiffre_affaire=21920000000, capitalisation_boursiere=66840000000),
 Resume(societe='Schneider Electric', chiffre_affaire=27200000000, capitalisation_boursiere=59830000000),
 Resume(societe='Airbus', chiffre_affaire=70478000000, capitalisation_boursiere=57840000000),
 Resume(societe='Vinci', chiffre_affaire=48753000000, capitalisation_boursiere=49950000000),
 Resume(societe='Essi

# Exercice d'application

Même questions pour le dow jones à partir de https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average

In [34]:
# structuration de la première étape
@dataclass
class Lien:
    societe: str
    adresse: str

In [36]:
def recupere_liens(
    adresse: str="https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
) -> list[Lien]:
    page = get(adresse)
    soupe = BeautifulSoup(page.content.decode("utf8"), "lxml")
    table = soupe.find(name="table", id="constituents")
    header, *lignes = table.find_all(name="tr")
    liens = [ligne.th.a for ligne in lignes]
    return [
        Lien(
            societe=lien.string,
            adresse="https://en.wikipedia.org/" + lien.attrs["href"]
        )
        for lien in liens
    ]

In [37]:
liens = recupere_liens()

In [38]:
@serialize
@deserialize
@dataclass 
class Page:
    societe: str
    adresse: str
    code_page: str

In [39]:
def conversion(lien: Lien) -> Page:
    page = get(lien.adresse)
    return Page(
        societe=lien.societe,
        adresse=lien.adresse,
        code_page=page.content.decode("utf8"),
    )

In [40]:
# Génération d'un fichier de sauvegarde brut
# taille effective: 14.5 mo
pages = [conversion(lien) for lien in liens]
with open("backup.json", "w") as fichier:
    fichier.write(to_json(pages))

In [42]:
# Récupération du backup
with open("backup.json", "r") as fichier:
    data = fichier.read()

pages = from_json(list[Page], data)

In [ ]:
# A implémenter
def recupere_info(page: Page) -> Resume:
    page = get(lien.adresse)
    soupe = BeautifulSoup(page.content.decode("utf8"), "lxml")
    ...

In [ ]:
# Conclusion
dow_jones = [recupere_info(lien) for lien in recupere_liens()]